You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of the importance of Artificial Intelligence in today's technology-driven world
- Mention of the rapid advancements and innovations in the field

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning algorithms
- Natural language processing and sentiment analysis
- AI-powered chatbots and virtual assistants
- Autonomous vehicles and robotics

2. Key Players in 

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become increasingly essential in today's technology-driven world, revolutionizing various industries with its capabilities. The rapid advancements and innovations in AI have paved the way for groundbreaking applications, making it a pivotal area of interest for tech enthusiasts and professionals alike.

## Latest Trends in Artificial Intelligence

The latest trends in AI encompass a wide range of developments, from machine learning and deep learning algorithms to natural language processing and sentiment analysis. These technologies are driving significant advancements in AI applications, enabling AI-powered chatbots, virtual assistants, autonomous vehicles, and robotics to thrive in various sectors.

AI is continuously evolving, with new trends emerging to shape the future of technology. Embracing these trends can lead

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become increasingly essential in today's technology-driven world, revolutionizing various industries with its capabilities. The rapid advancements and innovations in AI have paved the way for groundbreaking applications, making it a pivotal area of interest for tech enthusiasts and professionals alike.

## Latest Trends in Artificial Intelligence

The latest trends in AI encompass a wide range of developments, from machine learning and deep learning algorithms to natural language processing and sentiment analysis. These technologies are driving significant advancements in AI applications, enabling AI-powered chatbots, virtual assistants, autonomous vehicles, and robotics to thrive in various sectors.

AI is continuously evolving, with new trends emerging to shape the future of technology. Embracing these trends can lead to enhanced efficiency, improved decision-making processes, and innovative solutions in a world increasingly reliant on AI-driven systems.

## Key Players in the AI Industry

The AI industry boasts an array of key players that have significantly contributed to its growth and development. Companies like Google's DeepMind, IBM Watson, Amazon Web Services (AWS) AI, Microsoft Azure AI, OpenAI, and NVIDIA are at the forefront of AI innovation, driving progress and setting benchmarks for excellence in the field.

These key players invest heavily in research and development to push the boundaries of AI capabilities, creating cutting-edge solutions and services that cater to diverse market needs. Their contributions not only shape the industry landscape but also inspire future generations of AI enthusiasts to explore the endless possibilities of artificial intelligence.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research have captured the attention of the global community, showcasing the potential of AI to revolutionize various industries. From healthcare to finance, AI's impact is felt across sectors, offering new opportunities for growth, efficiency, and innovation.

As AI continues to advance, ethical considerations and regulations have become crucial topics of discussion. The responsible development and deployment of AI technologies are essential to ensure their benefits are maximized while mitigating potential risks and challenges associated with their use.

## Conclusion

In conclusion, Artificial Intelligence remains a dynamic and rapidly evolving field, with the latest trends, key players, and noteworthy news shaping its trajectory. By staying informed about AI developments and innovations, individuals can leverage the power of AI to drive progress, foster innovation, and create a better future for all.

As we navigate the ever-changing landscape of AI, it is essential to engage with the latest trends, understand the key players driving innovation, and stay informed about the significant news shaping the industry. By embracing AI's potential and exploring its practical applications, we can unlock new opportunities and possibilities that will shape the future of technology and society.

**Call to Action:**
Explore more about Artificial Intelligence by subscribing to our newsletter or attending AI-related events. Share your thoughts and experiences with AI in the comments section below.

Remember, the future is AI, and by staying informed and engaged, we can harness its power to drive positive change and innovation in the world.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
#topic = "YOUR TOPIC HERE"
topic = "machine learning"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on machine learning.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: The Latest Trends in Machine Learning: A Comprehensive Guide

Introduction:
- Definition of machine learning
- Importance of staying updated on trends in machine learning

Key Points:
1. Latest Trends in Machine Learning:
- Advancements in deep learning algorithms
- Increased focus on interpretability and explainability in machine learning models
- Growth of automated machine learning tools
- Adoption of reinforcement learning in various industries
- Integration of machine learning wit

I now can give a great answer

Final Answer:

# The Latest Trends in Machine Learning: A Comprehensive Guide

## Introduction
Machine learning, a subset of artificial intelligence, has been revolutionizing industries with its ability to analyze data and make predictions without explicit programming. Staying updated on the latest trends in machine learning is crucial for professionals in the field to remain competitive and innovative.

## Latest Trends in Machine Learning
Advancements in deep learning algorithms have been at the forefront of the latest trends in machine learning. These algorithms, inspired by the structure and function of the human brain, have enabled breakthroughs in image recognition, natural language processing, and speech recognition. Additionally, there has been an increased focus on interpretability and explainability in machine learning models, as stakeholders seek to understand the decision-making processes of these complex systems.

The growth of automated mach

In [14]:
Markdown(result)

# The Latest Trends in Machine Learning: A Comprehensive Guide

## Introduction
Machine learning, a subset of artificial intelligence, has been revolutionizing industries with its ability to analyze data and make predictions without explicit programming. Staying updated on the latest trends in machine learning is crucial for professionals in the field to remain competitive and innovative.

## Latest Trends in Machine Learning
Advancements in deep learning algorithms have been at the forefront of the latest trends in machine learning. These algorithms, inspired by the structure and function of the human brain, have enabled breakthroughs in image recognition, natural language processing, and speech recognition. Additionally, there has been an increased focus on interpretability and explainability in machine learning models, as stakeholders seek to understand the decision-making processes of these complex systems.

The growth of automated machine learning tools has democratized the field, allowing individuals with limited coding experience to build and deploy machine learning models. Moreover, the adoption of reinforcement learning in various industries, such as finance and healthcare, has led to significant improvements in decision-making and optimization. Furthermore, the integration of machine learning with Internet of Things (IoT) devices has enabled the development of smart systems that can learn and adapt to their environments.

## Key Players in the Machine Learning Industry
Several key players in the machine learning industry are leading the way with innovative technologies and platforms. Google, with TensorFlow and Cloud Machine Learning Engine, has been a pioneer in the development of open-source machine learning frameworks. Microsoft offers Azure Machine Learning and Cognitive Services, empowering organizations to build intelligent applications with ease. Amazon's Amazon SageMaker and Rekognition have transformed how companies leverage machine learning for various use cases. IBM's Watson Machine Learning and IBM Cloud provide robust solutions for enterprise-scale machine learning projects. Facebook, with PyTorch and Prophet, continues to push the boundaries of machine learning research and applications.

## Noteworthy News in Machine Learning
Keeping up with the latest news in machine learning is essential for professionals in the field. Recent studies and research findings have explored topics such as ethical considerations in AI and the impact of bias in machine learning algorithms. Acquisitions and partnerships among major companies in the machine learning industry have reshaped the competitive landscape, driving innovation and collaboration. Updates on regulations and ethical considerations in machine learning highlight the importance of responsible AI development and deployment.

In conclusion, the world of machine learning is evolving rapidly, with advancements in deep learning algorithms, the growth of automated machine learning tools, and the integration of machine learning with IoT devices shaping the future of the industry. Key players like Google, Microsoft, Amazon, IBM, and Facebook are driving innovation and setting the standard for excellence in machine learning technologies. Staying informed on the latest trends and news in machine learning is essential for professionals looking to stay ahead in this dynamic and exciting field.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).